# Decision Tree

Now that we have our possibilities matrix, can we construct a decision tree of depth 6 or smaller for each dictionary word?

In [1]:
import numpy as np
import pandas as pd

In [2]:
# make sure we reload the code
%load_ext autoreload
%autoreload 2

In [3]:
# we have to use some previous code
import sys
sys.path.append('..')

In [4]:
from parse_data import read_parsed_words, read_all_answers
from possibilities_table import array_to_integer, TABLE_PATH, integer_to_arr
from play import UNSAFE_eval_guess, eval_guess, LETTER_ABSENT

In [5]:
# words = read_parsed_words()
words = [w.lower() for w in read_all_answers()]
len(words)

2315

In [6]:
# table = np.load(TABLE_PATH)
TABLE_PATH_CHEATING = "../data-parsed/possibilities-table-cheating-base-3.npy"
table = np.load(TABLE_PATH_CHEATING)
table

array([[242,  81,   6, ..., 108, 135,  27],
       [  1, 242,   0, ...,  82,  29,   3],
       [  6,   0, 242, ..., 226,   0,  38],
       ...,
       [  4,  12, 216, ..., 242,   4,  28],
       [ 64,  14,   0, ...,  37, 242,  27],
       [  9,  81,   2, ...,  10,   9, 242]], dtype=uint8)

In [7]:
table.shape

(2315, 2315)

In [8]:
df = pd.DataFrame(table, index=words, columns=words)
df

,cigar,rebut,sissy,humph,awake,blush,focal,evade,naval,serve,...,rival,untie,refit,aorta,adult,judge,rower,artsy,rural,shave
cigar,242,81,6,0,27,0,55,27,54,81,...,141,3,84,108,27,9,162,108,135,27
rebut,1,242,0,27,3,36,0,3,0,7,...,2,111,170,82,189,30,5,82,29,3
sissy,6,0,242,0,0,64,0,0,0,38,...,6,3,3,0,0,0,0,226,0,38
humph,0,3,0,242,0,166,0,0,0,0,...,0,3,0,0,3,6,0,0,6,82
awake,10,81,0,0,242,0,10,181,10,162,...,10,162,81,11,11,162,84,11,10,181
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
judge,27,84,0,6,162,3,0,171,0,162,...,0,165,81,0,12,242,81,0,6,162
rower,163,110,0,0,36,0,6,27,0,109,...,83,27,110,88,0,27,242,82,83,27
artsy,4,12,216,0,2,54,1,1,1,30,...,4,18,12,14,11,0,3,242,4,28
rural,64,14,0,6,27,84,216,27,216,19,...,227,3,11,46,111,6,11,37,242,27


Use the mean partition size to select most promising words as first guesses first

In [9]:
def get_mean_partition(row) -> int:
    x = row.value_counts().to_dict()
    arr = np.array([v for v in x.values()])
    m = np.mean(arr)
    return m

mean_part_df = df.apply(get_mean_partition, axis=1)
mean_part_df = pd.DataFrame(mean_part_df, columns=['mean_partition'])
mean_part_df['word_index'] = np.arange(len(words))
mean_part_df.sort_values(by='mean_partition')

,mean_partition,word_index
trace,15.433333,179
crate,15.641892,45
slate,15.748299,878
parse,15.856164,1640
crane,16.302817,1729
...,...,...
kappa,70.151515,1387
mamma,72.343750,2137
jazzy,74.677419,1739
kayak,74.677419,2100


map from a word's index to it's "score" -> lower is better

In [10]:
score_dict = mean_part_df.set_index('word_index').to_dict()['mean_partition']
# these are indexes
sorted_guesses = sorted(np.arange(len(words)), key=lambda i: score_dict[i])

## Benchmarking Methods

find_possible_answers is a function that we call very often. We need to make it as fast as possible

### find_possible_answers

In [12]:
from tester import find_possible_answers, NEW_find_possible_answers

In [13]:
# let's use these parameters in testing
guesses_w = ["slate", "pudgy"]
guesses = [words.index(i) for i in guesses_w]
guess_results = [0, 3]

In [14]:
%timeit -n 1000 find_possible_answers(guesses, guess_results, table)

265 µs ± 18 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [15]:
%timeit -n 1000 NEW_find_possible_answers(guesses, guess_results, table)

5.63 ms ± 162 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


#### results

`find_possible_answers` is about 20x faster than `NEW_find_possible_answers`

### pick_next_guesses_it

In [132]:
words = read_parsed_words()
# words = [w.lower() for w in read_all_answers()]
len(words)

12972

In [134]:
table = np.load(TABLE_PATH)
# TABLE_PATH_CHEATING = "../data-parsed/possibilities-table-cheating-base-3.npy"
# table = np.load(TABLE_PATH_CHEATING)

In [135]:
df = pd.DataFrame(table, index=words, columns=words)
df

,aahed,aalii,aargh,aarti,abaca,abaci,aback,abacs,abaft,abaka,...,zulus,zupan,zupas,zuppa,zurfs,zuzim,zygal,zygon,zymes,zymic
aahed,242,8,17,8,5,5,5,5,5,5,...,0,4,4,4,0,0,4,0,54,0
aalii,8,242,8,197,5,194,5,5,5,5,...,18,4,4,4,0,135,13,0,0,135
aargh,89,8,242,26,5,5,5,5,5,5,...,0,4,4,4,18,0,31,27,0,0
aarti,8,170,26,242,5,167,5,5,32,5,...,0,4,4,4,18,81,4,0,0,81
abaca,92,92,92,92,242,161,161,161,107,188,...,0,91,91,172,0,0,91,0,0,27
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
zuzim,0,54,0,27,0,27,0,0,0,0,...,17,17,17,17,17,242,11,11,92,146
zygal,27,108,36,27,27,27,27,27,27,27,...,83,56,56,29,2,2,242,26,8,8
zygon,0,0,9,0,0,0,0,0,0,0,...,2,164,2,2,2,2,26,242,8,8
zymes,54,0,0,0,0,0,0,162,0,0,...,164,2,164,2,164,11,8,8,242,26


In [136]:
def get_mean_partition(row) -> int:
    x = row.value_counts().to_dict()
    arr = np.array([v for v in x.values()])
    m = np.mean(arr)
    return m

mean_part_df = df.apply(get_mean_partition, axis=1)
mean_part_df = pd.DataFrame(mean_part_df, columns=['mean_partition'])
mean_part_df['word_index'] = np.arange(len(words))
mean_part_df.sort_values(by='mean_partition')

,mean_partition,word_index
tares,61.188679,11115
teras,62.066986,11249
tears,63.588235,11183
pelas,64.217822,8095
pares,64.537313,7975
...,...,...
cocco,360.333333,2157
queue,370.628571,8780
taata,370.628571,11012
ayaya,381.529412,713


In [137]:
# these are indexes
sorted_guesses = mean_part_df.sort_values('mean_partition').word_index.values
sorted_guesses

array([11115, 11249, 11183, ..., 11012,   713,  8742])

In [138]:
# let's use these parameters in testing
guesses_w = ["crane", "polis"]
guesses = [words.index(i) for i in guesses_w]
guess_results = [0, 168]

In [139]:
possible_answers = find_possible_answers(guesses, guess_results, table)

In [140]:
from decision_tree import pick_next_guesses_it, NEW_pick_next_guesses_it

In [195]:
def f1():
    n = 0
    next_guesses_it = pick_next_guesses_it(guesses, guess_results, sorted_guesses, words)
    for guess in next_guesses_it:
        n += 1
        
%timeit f1()

11.7 ms ± 686 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [197]:
def f2():
    n = 0
    next_guesses_it = NEW_pick_next_guesses_it(guesses, guess_results, table, possible_answers)
    for guess in next_guesses_it:
        n += 1
        
%timeit f2()

396 ms ± 10.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [143]:
table

array([[242,   8,  17, ...,   0,  54,   0],
       [  8, 242,   8, ...,   0,   0, 135],
       [ 89,   8, 242, ...,  27,   0,   0],
       ...,
       [  0,   0,   9, ..., 242,   8,   8],
       [ 54,   0,   0, ...,   8, 242,  26],
       [  0,  54,   0, ...,   8,  26, 242]], dtype=uint8)

In [200]:
df = pd.DataFrame(table, index=words, columns=words)
df

,aahed,aalii,aargh,aarti,abaca,abaci,aback,abacs,abaft,abaka,...,zulus,zupan,zupas,zuppa,zurfs,zuzim,zygal,zygon,zymes,zymic
aahed,242,8,17,8,5,5,5,5,5,5,...,0,4,4,4,0,0,4,0,54,0
aalii,8,242,8,197,5,194,5,5,5,5,...,18,4,4,4,0,135,13,0,0,135
aargh,89,8,242,26,5,5,5,5,5,5,...,0,4,4,4,18,0,31,27,0,0
aarti,8,170,26,242,5,167,5,5,32,5,...,0,4,4,4,18,81,4,0,0,81
abaca,92,92,92,92,242,161,161,161,107,188,...,0,91,91,172,0,0,91,0,0,27
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
zuzim,0,54,0,27,0,27,0,0,0,0,...,17,17,17,17,17,242,11,11,92,146
zygal,27,108,36,27,27,27,27,27,27,27,...,83,56,56,29,2,2,242,26,8,8
zygon,0,0,9,0,0,0,0,0,0,0,...,2,164,2,2,2,2,26,242,8,8
zymes,54,0,0,0,0,0,0,162,0,0,...,164,2,164,2,164,11,8,8,242,26


In [210]:
possible_answers_arr = np.array(list(possible_answers))

In [211]:
m = table[:, possible_answers_arr]

In [223]:
m_out = np.unique(m, axis=1)

In [227]:
set(m_out[0])

{0, 9, 18, 81, 90}

In [229]:
np.unique(m[0])

array([ 0,  9, 18, 81, 90], dtype=uint8)

In [230]:
m_out, counts = np.unique(m, axis=1, return_counts=True)

In [231]:
m_out

array([[  0,   0,   0, ...,  81,  90,  90],
       [  0,   0,   0, ...,   0,   0,   0],
       [  0,   0,   0, ...,  27,  81,  81],
       ...,
       [ 27,  27,  27, ...,  36,  27,  30],
       [162, 162, 180, ..., 162, 162, 165],
       [  0,   0,  18, ...,   0,   0,   3]], dtype=uint8)

In [232]:
set(m_out[0])

{0, 9, 18, 81, 90}

In [233]:
counts

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [239]:
# how about an example

m_sample = np.array([
    [1, 2, 1],
    [2, 2, 2],
    [3, 3, 3],
    [1, 4, 5],
])
m_sample

array([[1, 2, 1],
       [2, 2, 2],
       [3, 3, 3],
       [1, 4, 5]])

In [242]:
np.unique(m_sample, axis=1, return_counts=True)

(array([[1, 1, 2],
        [2, 2, 2],
        [3, 3, 3],
        [1, 5, 4]]),
 array([1, 1, 1]))

In [243]:
np.unique(m_sample, axis=0, return_counts=True)

(array([[1, 2, 1],
        [1, 4, 5],
        [2, 2, 2],
        [3, 3, 3]]),
 array([1, 1, 1, 1]))

In [209]:
def my_pick_next_guesses_it(guesses, guess_results, table, possible_answers):
    possible_answers_arr = np.array(list(possible_answers))
    m = table[:, possible_answers_arr]
    sort_arr = np.apply_along_axis(get_mean_partition, axis=1, arr=m)
    pass


In [208]:
%timeit df.apply(get_mean_partition, axis=0)
# v

5.31 s ± 181 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


Fastest way to create numpy array from set

In [154]:
%timeit np.array(list(possible_answers))

8.79 µs ± 161 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [155]:
%timeit np.array([answer for answer in possible_answers])

10.8 µs ± 101 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [156]:
pas = np.array(list(possible_answers))
m1 = table[:, pas]


In [168]:
%timeit np.unique(m1[0])

5.59 µs ± 94.6 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [167]:
%timeit set(m1[0])

11.3 µs ± 311 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [175]:
%timeit np.bincount(m1[0]).max()

3.7 µs ± 71.6 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [176]:
def f2():
    _, counts = np.unique(m1[0], return_counts=True)
    return counts.max()

%timeit f2

15.8 ns ± 0.294 ns per loop (mean ± std. dev. of 7 runs, 100000000 loops each)


In [180]:
def worst_partition_unique(row):
    _, counts = np.unique(row, return_counts=True)
    return counts.max()

%timeit -n 5 np.apply_along_axis(worst_partition_unique, axis=1, arr=table)

KeyboardInterrupt: 

In [179]:
def worst_partition_bincount(row):
    return np.bincount(row).max()

%timeit -n 10 np.apply_along_axis(worst_partition_bincount, axis=1, arr=table)

371 ms ± 3.51 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [194]:
def get_mean_partition(row: np.ndarray) -> float:
    _, counts = np.unique(row, return_counts=True)
    return np.mean(counts)

%timeit np.apply_along_axis(get_mean_partition, axis=1, arr=table)

3.4 s ± 60.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


#### Results

`pick_next_guesses_it` based on a static heuristic is about 38x faster than our more sophisticated path for computing the heuristic. I just hope it isn't 38x worse.

## Checking the Generated Decision Tree

I generated some decision trees using the answers dictionary. I think the program would take a monstrous amount of time to run if I tried running it on the full dictionary.

### Answers Only

It takes about 2 minutes to run this program for a given root word without optimizations.

- With optimization #1 only: ~40s
- With optimizations #1 and #2: rebut, 5.4s, 5,341 states opened
    - slate: 6.3s, 7,880 states opened
    - crane: 11.8s, 21,762 states opened
- With optimizations #1, #2, #3: ~18s, 5595 states opened (fewer states but slower)

Optimization #3 is not very good because the method takes about 40x longer to compute, but on average saves us very few states, since often we recurse on 1 state.

In [84]:
import json

In [90]:
ALL_LETTERS_CORRECT = (3 ** 5) - 1
ALL_LETTERS_CORRECT

242

In [91]:
IS_DEBUG = False

def print_debug(s):
    if IS_DEBUG:
        print(s)

In [99]:
def find_answer(answer: str, tree: dict, depth: int) -> int:
    # get the top-level key
    root_words = list(tree.keys())
    assert len(root_words) == 1
    root_word = root_words[0]
    guess = words[int(root_word)]
    print_debug(f'depth {depth}, guessing {guess}')
    
    rv = eval_guess(guess, answer)
    print_debug(rv)
    rvi = array_to_integer(rv)
    if rvi == 242:
        print_debug('Correct')
        return depth
    
    action_map = tree[root_word]
    if str(rvi) in action_map:
        return find_answer(
            answer=answer,
            tree=action_map[str(rvi)],
            depth=depth+1
        )

In [116]:
def check_tree_answers(path: str):
    tree = {}
    with open(path) as fp:
        tree = json.load(fp)
        
    words = [w.lower() for w in read_all_answers()]
    # what is the root word?
    root_word = list(tree.keys())[0]
    print('root word is', words[int(root_word)])
    
    
    d = {}

    for answer in words:
        depth = find_answer(answer, tree, depth=1)
        d[answer] = depth
        
    ws = [w for w in d.keys()]
    vs = [v for v in d.values()]

    dt_df = pd.DataFrame({
        'word': ws,
        'depth': vs
    })
    display(dt_df)
    
    mean = dt_df.depth.mean()
    print(f'mean {mean:.2f}')
    return dt_df

In [130]:
path = '../out/decision-trees/answers/paper.json'

In [131]:
dt_df = check_tree_answers(path)

root word is paper


,word,depth
0,cigar,4
1,rebut,4
2,sissy,5
3,humph,4
4,awake,6
...,...,...
2310,judge,6
2311,rower,6
2312,artsy,3
2313,rural,5


mean 4.49


So obviously this is not the optimal depth for this decision tree, but it always solves the puzzle

In [129]:
dt_df[dt_df.depth > 6]

,word,depth


## Solving the Full Dictionary

Now that solving with the answers dictionary is trivial, we can try using the full dictionary.

Optimizations #1 and #2 are must-haves and are very powerful. If only these two are on, then the initial path chosen looks like:

> crane -> 0 -> toils

To get to iteration 118, it takes about 30 seconds. That one looks like:

> crane -> 0 -> toils -> 162

I profiled the program while running it this way for about a minute and a half. A large fraction of the time is spent:

- 20/98 -> `pick_next_guess_it`
    - most of that in filtering out guesses using black letters
    - 99740 calls
    - 18687614 calls to the listcomp looking for black letters
    - just as many calls to `any`
- 15/98 -> `np.where`
    - 2555504 calls
    - this is only used in the computation of possible answers
- 6/98 -> `np.unique`
    - this is only used to find the unique values for a guess
    - 31936 calls
- 6/98 -> `np.sort`
    - I have no idea where this is called...
    - I think it's being called by unique, because it's called exactly 1 time more than unique

### new optimizations

These optimizations aren't numbered because I don't see why you wouldn't use them.

- Opt 5: early exit on failed subtree. When, for a given result, no further guess is able to solve that subtree, we know that we don't need to look any further for other guess results. We have failed and can early exit from the guess_result loop

- Opt 6: sort guess results by partition size. When we iterate over guess results, we shouldn't iterate over them in some arbitrary order, but should try the cases that are most likely to fail first.

### enabling optimization #4

The program runs noticeably faster with optimization #4 enabled.It looks like we are able to choose better paths at higher levels and avoid entire subtrees.

Optimization #4 uses a heuristic to pick the guess which creates the smallest mean partition size (thus optimizing for a large number of small partitions). This optimization is only enabled at levels 1 and 2 (when choosing guesses 2 and 3) and not at the lower levels. The function that computes it is fairly expensive (see above).

First slow path (iteration 125, takes about 50 seconds to get there):

> crane -> 0 -> polis -> 168

I killed the program before that path is fully resolved.

When using a profiler, the hot paths have changed. We are using a lot of `np.where` and `np.unique` as before, but `pick_next_guess_it` is no longer called as many times (only 372,582). I think this shows that pruning at higher levels is very effective.

- most time is spent in `NEW_pick_next_guesses_it` with `apply_along_axis` taking the longest amount of time

The runtime seems to be dominated by calls to `get_mean_partition`: 1,089,648 of them

### optimization #4 - switching to worst_partition

I found a very efficient way to write `get_worst_partition` using the numpy method `bincount`. What if we use that as the heuristic instead of `get_mean_partition`?

Instead of picking `polis` for a second word, we pick `molts`. But we're moving very quickly through the possible values. That was a lie no we're not.

So now we're back to `get_mean_partition`

### optimization #3

I did not previously enable optimization #3 because it made the program much slower. However, I've found a way to fix it so it's much faster.

The idea is, once we get down to depth 4 (choosing our 5th guess), we can pick either the optimal word to partition the space such that all partitions have only 1 possible answer, or we're not going to solve that subtree, so we can exit early.

This saves on exploring and failing to solve a bunch of stuff at depth 4.

Once we have this optimization enabled, we are able to progress much further.

First slow path (iteration 40, takes about 30s to get there):

> crane -> 0 -> polis -> 207

### optimization #7

We can save bad paths between runs. We know those paths don't work out. Don't explore them.